# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Heqing_device2"

TIME_WINDOW = "30s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [3]:
import sys
repo_base_path = Path("./../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [4]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [5]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
filetype
csv    530
zip     92
Name: count, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
KERN_data       91
FLS_data        91
RES_data        91
block_data      91
network_data    91
entropy_data    75
Name: count, dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [6]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(133, 9)


### Get First TimeStamp

In [7]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

! Dropped rows in DataFrames: (2,)
time                                                time
cpu                                                  cpu
seconds                                          seconds
ioread                                            ioread
iowrite                                          iowrite
                                         ...            
rpm:rpm_return_int                    rpm:rpm_return_int
fib:fib_table_lookup                fib:fib_table_lookup
raw_syscalls:sys_enter            raw_syscalls:sys_enter
random:credit_entropy_bits    random:credit_entropy_bits
timestamp                                            NaN
Name: 318, Length: 118, dtype: object
time                                                time
cpu                                                  cpu
seconds                                          seconds
ioread                                            ioread
iowrite                                          iowrite
               

In [8]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-11-09 15:18:45 and 2023-11-13 02:07:23'

### Resample files

In [9]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [10]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

Data type: DataSource.FileSystem
Shape: (198, 54)
Data type: DataSource.FileSystem
Shape: (315, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (126, 54)
Data type: DataSource.FileSystem
Shape: (180, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
Data type: DataSource.FileSystem
Shape: (242, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (241, 54)
Data type: DataSource.FileSystem
Shape: (240, 54)
same_df shape: (4368, 57)
Data type: DataSource.Ke

5

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [11]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

Reindexing only valid with uniquely valued Index objects


In [12]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['writeback:writeback_dirty_inode_enqueue', 'writeback:writeback_dirty_inode', 'block:block_bio_backmerge', 'writeback:sb_clear_inode_writeback', 'block:block_bio_remap', 'filemap:mm_filemap_add_to_page_cache', 'jbd2:jbd2_handle_start', 'writeback:writeback_mark_inode_dirty', 'jbd2:jbd2_checkpoint_stats', 'ext4:ext4_unlink_enter', 'jbd2:jbd2_start_commit', 'block:block_getrq', 'ext4:ext4_es_lookup_extent_enter', 'writeback:global_dirty_state', 'writeback:writeback_dirty_page', 'block:block_unplug', 'filemap:mm_filemap_delete_from_page_cache', 'block:block_dirty_buffer', 'jbd2:jbd2_handle_stats', 'ext4:ext4_free_inode', 'ext4:ext4_da_update_reserve_space', 'seconds', 'ext4:ext4_writepages_result', 'ext4:ext4_releasepage', 'block:block_touch_buffer', 'ext4:ext4_journal_start', 'label', 'device', 'data_source', 'page-faults', 'kmem:mm_page_free', 'kmem:kfree', 'timer:timer_start', 'gpio:gpio_value', 'sched:sched_process_exec', 'task:task_newtask', 'fib:fib_table_lookup', 'kmem

/tmp/ipykernel_56078/3047974697.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  infos = pd.concat([infos,dfs[to_ignore]], axis=0)


In [13]:
df_all = pd.concat(df_mean_collection[:-1], axis=1).merge(df_mean_collection[-1], how='outer', left_index=True, right_index=True)
df_all.sample(2)

,ext4:ext4_truncate_enter,ext4:ext4_da_write_end,writeback:writeback_dirty_inode_enqueue_FLS_data,writeback:writeback_dirty_inode_FLS_data,block:block_bio_backmerge_FLS_data,jbd2:jbd2_handle_extend,ext4:ext4_es_insert_delayed_block,writeback:sb_clear_inode_writeback_FLS_data,block:block_bio_remap_FLS_data,filemap:mm_filemap_add_to_page_cache_FLS_data,...,tcp:tcp_probe_RES_data,armv7_cortex_a15/l2d_cache/,instructions,armv7_cortex_a15/l1i_cache/,context-switches,write_ops,util,write_kbs,(OLD) VarianceLength,(OLD) AverageLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 22:00:45,24.0,105.0,30.0,1012.0,17.0,0.0,25.0,55.0,62.0,24.0,...,1349.5,668292786.5,1.427817e+10,7.056708e+09,26829.0,32.233333,5.253333,2.8,495627.395799,815.530501
2023-11-10 20:53:15,0.0,0.0,2.0,1362.0,2.0,0.0,0.0,1.0,5.0,0.0,...,2129.0,593045534.5,1.899423e+10,8.591424e+09,18738.0,4.400000,8.800000,0.6,17609.968393,114.176471


In [14]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [15]:
df_all.shape, len(SELECTED_FEATURES)

((23315, 205), 260)

In [16]:
df_all

,ext4:ext4_truncate_enter,ext4:ext4_da_write_end,writeback:writeback_dirty_inode_enqueue_FLS_data,writeback:writeback_dirty_inode_FLS_data,block:block_bio_backmerge_FLS_data,jbd2:jbd2_handle_extend,ext4:ext4_es_insert_delayed_block,writeback:sb_clear_inode_writeback_FLS_data,block:block_bio_remap_FLS_data,filemap:mm_filemap_add_to_page_cache_FLS_data,...,armv7_cortex_a15/l2d_cache/,instructions,armv7_cortex_a15/l1i_cache/,context-switches,write_ops,util,write_kbs,(OLD) VarianceLength,(OLD) AverageLength,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,5.354512e+08,1.603289e+10,8.628315e+09,19407.0,3.333333,11.000000,0.5,332853.277927,1194.224582,1_normal
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,5.354512e+08,1.603289e+10,8.628315e+09,19407.0,3.333333,11.000000,0.5,332853.277927,1194.224582,1_normal
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,5.354512e+08,1.603289e+10,8.628315e+09,19407.0,3.333333,11.000000,0.5,332853.277927,1194.224582,1_normal
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,5.354512e+08,1.603289e+10,8.628315e+09,19407.0,3.333333,11.000000,0.5,332853.277927,1194.224582,1_normal
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,5.354512e+08,1.603289e+10,8.628315e+09,19407.0,3.333333,11.000000,0.5,332853.277927,1194.224582,1_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 02:06:45,0.0,0.0,0.0,1306.0,4.0,0.0,0.0,0.0,6.0,0.0,...,5.989378e+08,1.710832e+10,9.906190e+09,263966.0,2.933333,9.333333,0.4,467462.369501,999.731343,9_xmrig
2023-11-13 02:06:45,0.0,0.0,0.0,1306.0,4.0,0.0,0.0,0.0,6.0,0.0,...,5.989378e+08,1.710832e+10,9.906190e+09,263966.0,2.933333,9.333333,0.4,467462.369501,999.731343,9_xmrig
2023-11-13 02:06:45,0.0,0.0,0.0,1306.0,4.0,0.0,0.0,0.0,6.0,0.0,...,5.989378e+08,1.710832e+10,9.906190e+09,263966.0,2.933333,9.333333,0.4,467462.369501,999.731343,9_xmrig


# Add Sys Call Data

In [17]:
def read_in_preprocessed_Heqing_device2():
    file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Heqing_device2/merged_data.npz')
    assert file_path.exists(), 'Data not found'

    with np.load(file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer()
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Heqing_device2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(11428, 162)
Loaded set: ('accept', 'accept4', 'access', 'add_key', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_create1', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fallocate', 'fchdir', 'fchmod', 'fchmodat', 'fchown32', 'fcntl64', 'fgetxattr', 'flistxattr', 'flock', 'fsetxattr', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getpriority', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'gettimeofday', 'getuid32', 'getxattr', 'inotify_add_watch', 'ioctl', 'ioprio_get', 'ioprio_set', 'kcmp', 'keyctl', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lseek', 'lstat64', 'madvise', 'mkdir', 'mknod', 'mmap2', 'mount', 'mpro

In [18]:
sys_df.head(2)

,accept,accept4,access,add_key,adjtimex,bind,brk,capget,capset,chdir,...,umount2,uname,unlink,unlinkat,utimensat,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.0,0.000008,0.000770,0.0,0.0,0.00002,0.001104,0.000037,0.00001,0.000006,...,0.0,0.000186,0.000006,0.0,0.0,0.000305,0.000023,0.028951,0.000006,1_normal
2023-11-09 15:18:55,0.0,0.000023,0.000481,0.0,0.0,0.00000,0.000726,0.000017,0.00000,0.000005,...,0.0,0.000176,0.000008,0.0,0.0,0.000231,0.000020,0.029970,0.000005,1_normal


In [19]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [20]:
sys_df_mean.head(2)

,exit_group,rt_sigaction,sched_yield,getsockname,sigreturn,fstat64,getxattr,epoll_create1,ugetrlimit,recvfrom,...,mmap2,prlimit64,statfs64,pause,fchmod,dup3,mprotect,readlink,send,sigaltstack
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.000226,0.002353,0.0,0.000032,0.000136,0.024232,0.001797,0.0,0.000209,0.000070,...,0.005148,0.000005,0.000016,0.000003,0.0,0.000002,0.002991,0.000020,0.000009,0.000009
2023-11-09 15:19:15,0.000285,0.002705,0.0,0.000047,0.000157,0.022755,0.002053,0.0,0.000182,0.000056,...,0.006072,0.000003,0.000023,0.000003,0.0,0.000002,0.003690,0.000019,0.000013,0.000013


In [21]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (28869, 306)


In [22]:
all_df1.head(4)

,ext4:ext4_truncate_enter,ext4:ext4_da_write_end,writeback:writeback_dirty_inode_enqueue_FLS_data,writeback:writeback_dirty_inode_FLS_data,block:block_bio_backmerge_FLS_data,jbd2:jbd2_handle_extend,ext4:ext4_es_insert_delayed_block,writeback:sb_clear_inode_writeback_FLS_data,block:block_bio_remap_FLS_data,filemap:mm_filemap_add_to_page_cache_FLS_data,...,mmap2,prlimit64,statfs64,pause,fchmod,dup3,mprotect,readlink,send,sigaltstack
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,0.005148,0.000005,0.000016,0.000003,0.0,0.000002,0.002991,0.00002,0.000009,0.000009
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,0.005148,0.000005,0.000016,0.000003,0.0,0.000002,0.002991,0.00002,0.000009,0.000009
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,0.005148,0.000005,0.000016,0.000003,0.0,0.000002,0.002991,0.00002,0.000009,0.000009
2023-11-09 15:18:45,0.0,0.0,5.0,1265.0,5.5,0.0,0.0,0.0,7.5,2.0,...,0.005148,0.000005,0.000016,0.000003,0.0,0.000002,0.002991,0.00002,0.000009,0.000009


In [23]:
spath = repo_base_path / "training" / "data" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))